In [4]:
### Import das bibliotecas necessarias
import pandas as pd
#import pyodbc
from sqlalchemy import create_engine

In [5]:
# Conexão com o banco de dados
try:
    server_name = 'DESKTOP-IBC02BG'
    database_name = 'dbProjetoImpacta'
    conn_str = f'mssql+pyodbc://{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
    engine = create_engine(conn_str)
    print('Conectado com sucesso!')
    
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Conectado com sucesso!


In [7]:
# API banco central cotação dolar
path = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2019'&@dataFinalCotacao='12-31-2025'&$top=9000&$format=text/csv&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
# Leitura dos dados e criação do dataframe
tabela = pd.read_csv(path)
# Renomeando as colunas do dataframe
tabela = tabela.rename(columns={'cotacaoCompra': 'Cotacao de Compra','cotacaoVenda':'Cotacao de Venda','dataHoraCotacao':'Data e Hora da Cotacao'})
# Separando a tabela data e hora em duas tabelas
tabela[['Data', 'Hora']] = tabela['Data e Hora da Cotacao'].str.split(' ', 1, expand=True)
# Apagando a coluna com data e hora juntos
tabela.drop(columns=['Data e Hora da Cotacao'], inplace=True)

In [12]:
# Converter os tipos e organizar os dados das colunas 

tabela['Cotacao de Compra'] = tabela['Cotacao de Compra'].astype(str).str.replace(',', '.').astype(float)
tabela['Cotacao de Venda'] = tabela['Cotacao de Venda'].astype(str).str.replace(',', '.').astype(float)
tabela['Data'] = pd.to_datetime(tabela['Data'], format='%Y-%m-%d')
tabela['Data'] = tabela['Data'].dt.strftime('%Y-%m-%d')
#tabela['Hora'] = pd.to_datetime(tabela['Hora'], format='%H:%M:%S.%f').dt.time
#tabela['Hora'] = pd.to_datetime(tabela['Hora'], format='%H:%M:%S.%f', errors='coerce')
tabela['Hora'] = pd.to_datetime(tabela['Hora'], format='%H:%M:%S.%f').dt.time

display(tabela)

,Cotacao de Compra,Cotacao de Venda,Data,Hora
0,3.8589,3.8595,2019-01-02,13:04:46.568000
1,3.7677,3.7683,2019-01-03,13:04:50.817000
2,3.7621,3.7627,2019-01-04,13:06:29.332000
3,3.7056,3.7062,2019-01-07,13:09:39.652000
4,3.7202,3.7208,2019-01-08,13:09:06.450000
...,...,...,...,...
1068,5.1248,5.1254,2023-03-30,13:11:18.582000
1069,5.0798,5.0804,2023-03-31,13:03:22.571000
1070,5.0631,5.0637,2023-04-03,13:04:18.827000
1071,5.0756,5.0762,2023-04-04,13:04:34.464000


In [10]:
print(tabela.dtypes)

Cotacao de Compra    object
Cotacao de Venda     object
Data                 object
Hora                 object
dtype: object


In [42]:
try:
    # Salvando o DataFrame no banco de dados MySQL
    tabela.to_sql('cotacao', con=engine, if_exists='replace', index=False)
    print('Dados gravados no banco de dados...')


except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados gravados no banco de dados...


In [6]:
# Executa a consulta SQL para contar as linhas da tabela
result = engine.execute('SELECT COUNT(*) FROM cotacao')
count = result.fetchone()[0]

print(f'O número de linhas na tabela é: {count}')

O número de linhas na tabela é: 1071


In [109]:
# Fechando a conexão com o banco de dados
engine.dispose()